In [4]:
# Финальный код для визуализации результатов и создания отчета
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os
import warnings
warnings.filterwarnings('ignore')

# Настройка стилей
plt.style.use('default')
sns.set_palette("husl")

# Определение путей к данным
# Проект: finish, Данные: finish\Data, Код: finish\notebooks\DeepSeek\finishdeepseek.ipynb
current_dir = os.path.abspath(os.getcwd())
BASE_DIR = current_dir

# Поднимаемся вверх, пока не найдем папку Data или не достигнем корня
max_levels = 5  # Максимальное количество уровней для подъема
for _ in range(max_levels):
    if os.path.exists(os.path.join(BASE_DIR, "Data")):
        break
    parent_dir = os.path.dirname(BASE_DIR)
    if parent_dir == BASE_DIR:  # Достигли корня файловой системы
        break
    BASE_DIR = parent_dir

# Если не нашли, используем фиксированный путь: поднимаемся на 2 уровня вверх
# от текущей директории (notebooks/DeepSeek -> notebooks -> finish)
if not os.path.exists(os.path.join(BASE_DIR, "Data")):
    BASE_DIR = os.path.abspath(os.path.join(current_dir, os.pardir, os.pardir))

DATA_RAW = os.path.join(BASE_DIR, "Data", "data_raw.csv")
FINAL_XLSX = os.path.join(BASE_DIR, "Data", "final_results_to_analyze.xlsx")

# Выводим информацию для отладки
print(f"📁 Текущая рабочая директория: {os.getcwd()}")
print(f"📁 Базовая директория проекта: {BASE_DIR}")
print(f"📁 Путь к данным: {DATA_RAW}")

def create_final_report():
    """Создает финальный отчет с визуализациями и результатами"""

    # Чтение данных
    df_raw = pd.read_csv(DATA_RAW)
    df_exp = pd.read_excel(FINAL_XLSX)
    
    # Преобразуем дату в datetime, если она еще не в этом формате
    if 'dt' in df_raw.columns:
        df_raw['dt'] = pd.to_datetime(df_raw['dt'])

    # Создание отчета в markdown
    report_content = []

    # Заголовок отчета
    report_content.append("# Анализ А/Б теста: Изменение расположения рекламного блока\n")

    # Часть 1: Дизайн эксперимента
    report_content.append("## Часть 1: Дизайн эксперимента\n")

    # 1.1. Изучение исторических данных
    report_content.append("### 1.1. Анализ пользовательского поведения до эксперимента\n")

    # Подготовка исторических данных
    # Проверяем структуру данных
    print(f"Столбцы в данных: {df_raw.columns.tolist()}")
    print(f"Типы событий: {df_raw['event_type'].unique() if 'event_type' in df_raw.columns else 'N/A'}")
    
    # Нормализуем названия событий
    if 'event_type' in df_raw.columns:
        df_raw['event_type'] = df_raw['event_type'].str.strip().str.lower()
    
    # Создаем бинарные колонки для каждого типа события
    df_raw['views'] = (df_raw['event_type'] == 'views').astype(int)
    df_raw['clicks'] = (df_raw['event_type'] == 'clicks').astype(int)
    df_raw['adds'] = (df_raw['event_type'] == 'adds').astype(int)
    
    # Строим воронку: наличие события за дату
    df_funnel = df_raw.groupby(['dt', 'client_id'], as_index=False).agg({
        'views': 'max',
        'clicks': 'max',
        'adds': 'max'
    })
    
    # Переименовываем для совместимости с остальным кодом
    df_funnel = df_funnel.rename(columns={'views': 'view', 'clicks': 'click', 'adds': 'add'})
    
    # Убеждаемся, что значения бинарные (0 или 1)
    for col in ['view', 'click', 'add']:
        df_funnel[col] = (df_funnel[col] > 0).astype(int)

    df_user = df_funnel.groupby('client_id').agg({
        'view': 'sum',
        'click': 'sum',
        'add': 'sum'
    }).reset_index()

    df_user['CTR'] = df_user.apply(lambda row: row['click'] / row['view'] if row['view'] > 0 else 0, axis=1)
    df_user['CR'] = df_user.apply(lambda row: row['add'] / row['click'] if row['click'] > 0 else 0, axis=1)

    # Визуализация 1: Распределение метрик
    fig1, axes1 = plt.subplots(2, 2, figsize=(15, 12))

    # Распределение просмотров
    axes1[0, 0].hist(df_user['view'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    axes1[0, 0].set_title('Распределение количества просмотров на пользователя', fontsize=12, fontweight='bold')
    axes1[0, 0].set_xlabel('Количество просмотров')
    axes1[0, 0].set_ylabel('Частота')
    axes1[0, 0].grid(True, alpha=0.3)

    # Распределение CTR
    axes1[0, 1].hist(df_user['CTR'][df_user['CTR'] > 0], bins=30, alpha=0.7, color='lightgreen', edgecolor='black')
    axes1[0, 1].set_title('Распределение CTR (пользователи с кликами)', fontsize=12, fontweight='bold')
    axes1[0, 1].set_xlabel('CTR')
    axes1[0, 1].set_ylabel('Частота')
    axes1[0, 1].grid(True, alpha=0.3)

    # Распределение CR
    axes1[1, 0].hist(df_user['CR'][df_user['CR'] > 0], bins=30, alpha=0.7, color='salmon', edgecolor='black')
    axes1[1, 0].set_title('Распределение CR (пользователи с добавлениями)', fontsize=12, fontweight='bold')
    axes1[1, 0].set_xlabel('CR')
    axes1[1, 0].set_ylabel('Частота')
    axes1[1, 0].grid(True, alpha=0.3)

    # Сравнение средних значений
    metrics_summary = df_user[['view', 'CTR', 'CR']].mean()
    bars = axes1[1, 1].bar(metrics_summary.index, metrics_summary.values,
                          color=['skyblue', 'lightgreen', 'salmon'], edgecolor='black')
    axes1[1, 1].set_title('Средние значения метрик на пользователя', fontsize=12, fontweight='bold')
    axes1[1, 1].set_ylabel('Среднее значение')
    axes1[1, 1].grid(True, alpha=0.3)

    # Добавление значений на столбцы
    for bar, value in zip(bars, metrics_summary.values):
        height = bar.get_height()
        axes1[1, 1].text(bar.get_x() + bar.get_width()/2., height + 0.001,
                        f'{value:.4f}', ha='center', va='bottom', fontweight='bold')

    plt.tight_layout()
    plt.savefig('historical_metrics.png', dpi=300, bbox_inches='tight')
    plt.close()

    report_content.append("![Исторические метрики](historical_metrics.png)\n")

    # Статистика по историческим данным
    global_ctr = df_user['click'].sum() / df_user['view'].sum()
    global_cr = df_user['add'].sum() / df_user['click'].sum() if df_user['click'].sum() > 0 else 0

    report_content.append(f"**Ключевые метрики до эксперимента:**")
    report_content.append(f"- Глобальный CTR: {global_ctr:.4f}")
    report_content.append(f"- Глобальный CR: {global_cr:.4f}")
    report_content.append(f"- Среднее количество просмотров на пользователя: {df_user['view'].mean():.2f}")
    report_content.append(f"- Всего пользователей в исторических данных: {len(df_user):,}\n")

    # 1.2. Расчет MDE и размера выборки
    report_content.append("### 1.2. Расчет MDE и размера выборки\n")

    def var_ratio(x, y):
        mean_x = np.mean(x)
        mean_y = np.mean(y)
        var_x = np.var(x, ddof=1)
        var_y = np.var(y, ddof=1)
        cov_xy = np.cov(x, y, ddof=1)[0][1]
        result = var_x/mean_y**2 + var_y*mean_x**2/mean_y**4 - 2*mean_x*cov_xy/mean_y**3
        return result

    def get_MDE(mu, std, sample_size, n_groups=2, target_share=0.5, r=1, alpha=0.05, beta=0.2):
        t_alpha = stats.norm.ppf(1 - (alpha / 2))
        t_beta = stats.norm.ppf(1 - beta)
        sample_ratio_correction = r + 2 + 1/r
        mde_abs = np.sqrt(sample_ratio_correction) * (t_alpha + t_beta) * std / np.sqrt(sample_size * (1 - target_share * (n_groups - 2)))
        mde_rel = mde_abs * 100 / mu
        return mde_abs, mde_rel

    # Расчет параметров
    views = df_user['view'].values
    clicks = df_user['click'].values
    ctr_variance = var_ratio(clicks, views)
    ctr_std = np.sqrt(ctr_variance)

    # Расчет MDE для разных размеров выборки
    sample_sizes = np.arange(5000, 50000, 5000)
    mde_table = []

    for size in sample_sizes:
        mde_abs, mde_rel = get_MDE(global_ctr, ctr_std, size, alpha=0.05, beta=0.2)
        mde_table.append({
            'sample_size': size,
            'mde_abs': mde_abs,
            'mde_rel': mde_rel
        })

    mde_df = pd.DataFrame(mde_table)

    # Визуализация 2: MDE vs Размер выборки
    fig2, ax2 = plt.subplots(figsize=(10, 6))
    ax2.plot(mde_df['sample_size'], mde_df['mde_rel'], marker='o', linewidth=2, markersize=6)
    ax2.axhline(y=2.5, color='r', linestyle='--', linewidth=2, label='Целевой эффект 2.5%')

    # Расчет максимальной выборки
    daily_users = df_funnel['client_id'].nunique()
    weekly_users = daily_users * 7
    max_sample = int(weekly_users * 0.2)
    mde_for_max, mde_rel_max = get_MDE(global_ctr, ctr_std, max_sample)

    ax2.axvline(x=max_sample, color='g', linestyle='--', linewidth=2, label=f'Макс. выборка ({max_sample:,})')
    ax2.plot(max_sample, mde_rel_max, 'ro', markersize=10)
    ax2.annotate(f'MDE: {mde_rel_max:.1f}%',
                xy=(max_sample, mde_rel_max),
                xytext=(max_sample+2000, mde_rel_max+0.5),
                arrowprops=dict(arrowstyle='->', color='red'))

    ax2.set_xlabel('Размер выборки', fontsize=12)
    ax2.set_ylabel('MDE (%)', fontsize=12)
    ax2.set_title('Зависимость MDE от размера выборки для CTR', fontsize=14, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_xticks(np.arange(0, 50001, 10000))

    plt.tight_layout()
    plt.savefig('mde_analysis.png', bbox_inches='tight')
    plt.close()

    report_content.append("![Анализ MDE](mde_analysis.png)\n")

    report_content.append("**Расчет параметров эксперимента:**")
    report_content.append(f"- Среднесуточные пользователи: {daily_users:,}")
    report_content.append(f"- Недельная аудитория: {weekly_users:,}")
    report_content.append(f"- Максимальная выборка (20%): {max_sample:,}")
    report_content.append(f"- MDE для максимальной выборки: {mde_rel_max:.2f}%")
    report_content.append(f"- Статус: {'✓ Достаточная мощность' if mde_rel_max <= 2.5 else '✗ Недостаточная мощность'}\n")

    # Часть 2: Результаты эксперимента
    report_content.append("## Часть 2: Результаты эксперимента\n")

    # 2.1. Базовая статистика по группам
    report_content.append("### 2.1. Распределение пользователей по группам\n")

    group_counts = df_exp['ab_group'].value_counts()
    report_content.append(f"- Контрольная группа: {group_counts.get('control', 0):,} пользователей")
    report_content.append(f"- Тестовая группа: {group_counts.get('test', 0):,} пользователей")
    report_content.append(f"- Общее количество: {len(df_exp):,} пользователей\n")

    # 2.2. Расчет метрик
    group_metrics = df_exp.groupby('ab_group').agg({
        'is_view_ads': 'sum',
        'cnt_view_ads': 'sum',
        'is_adds_ads': 'sum',
        'cnt_adds_ads': 'sum',
        'is_orders_ads': 'sum',
        'cnt_orders_ads': 'sum',
        'sum_orders_ads': 'sum',
        'client_id': 'nunique'
    }).reset_index()

    group_metrics['CTR'] = group_metrics['is_view_ads'] / group_metrics['client_id']
    group_metrics['CR'] = group_metrics['is_adds_ads'] / group_metrics['is_view_ads']
    group_metrics['ARPU'] = group_metrics['sum_orders_ads'] / group_metrics['client_id']
    group_metrics['ARPPU'] = group_metrics['sum_orders_ads'] / group_metrics['is_orders_ads']
    group_metrics['avg_order_value'] = group_metrics['sum_orders_ads'] / group_metrics['cnt_orders_ads']

    # 2.3. Статистические тесты
    def proportion_z_test(success_a, size_a, success_b, size_b):
        p1 = success_a / size_a
        p2 = success_b / size_b
        p_pool = (success_a + success_b) / (size_a + size_b)

        z = (p1 - p2) / np.sqrt(p_pool * (1 - p_pool) * (1/size_a + 1/size_b))
        p_value = 2 * (1 - stats.norm.cdf(abs(z)))

        return z, p_value

    def confidence_interval(success, size, confidence=0.95):
        p = success / size
        z = stats.norm.ppf(1 - (1 - confidence) / 2)
        margin = z * np.sqrt(p * (1 - p) / size)
        return p, p - margin, p + margin

    # Данные для тестов
    control_data = group_metrics[group_metrics['ab_group'] == 'control'].iloc[0]
    test_data = group_metrics[group_metrics['ab_group'] == 'test'].iloc[0]

    # CTR тест
    z_ctr, p_ctr = proportion_z_test(
        test_data['is_view_ads'], test_data['client_id'],
        control_data['is_view_ads'], control_data['client_id']
    )

    ctr_control, ctr_ci_low_c, ctr_ci_high_c = confidence_interval(
        control_data['is_view_ads'], control_data['client_id']
    )
    ctr_test, ctr_ci_low_t, ctr_ci_high_t = confidence_interval(
        test_data['is_view_ads'], test_data['client_id']
    )

    # CR тест
    z_cr, p_cr = proportion_z_test(
        test_data['is_adds_ads'], test_data['is_view_ads'],
        control_data['is_adds_ads'], control_data['is_view_ads']
    )

    # Визуализация 3: Сравнение ключевых метрик
    fig3, axes3 = plt.subplots(2, 3, figsize=(18, 12))

    metrics_to_plot = [
        ('CTR', ctr_control, ctr_test, ctr_ci_low_c, ctr_ci_high_c, ctr_ci_low_t, ctr_ci_high_t),
        ('CR', control_data['CR'], test_data['CR'],
         control_data['CR'] - (control_data['CR'] - confidence_interval(control_data['is_adds_ads'], control_data['is_view_ads'])[1]),
         control_data['CR'] + (confidence_interval(control_data['is_adds_ads'], control_data['is_view_ads'])[2] - control_data['CR']),
         test_data['CR'] - (test_data['CR'] - confidence_interval(test_data['is_adds_ads'], test_data['is_view_ads'])[1]),
         test_data['CR'] + (confidence_interval(test_data['is_adds_ads'], test_data['is_view_ads'])[2] - test_data['CR']))
    ]

    for idx, (metric_name, control_val, test_val, ci_low_c, ci_high_c, ci_low_t, ci_high_t) in enumerate(metrics_to_plot):
        # Основные метрики
        bars = axes3[0, idx].bar(['Control', 'Test'], [control_val, test_val],
                               color=['#1f77b4', '#ff7f0e'], alpha=0.7, edgecolor='black')
        axes3[0, idx].errorbar(['Control', 'Test'], [control_val, test_val],
                             yerr=[[control_val - ci_low_c, test_val - ci_low_t],
                                   [ci_high_c - control_val, ci_high_t - test_val]],
                             fmt='none', color='black', capsize=5)
        axes3[0, idx].set_title(f'{metric_name} с доверительными интервалами', fontweight='bold')
        axes3[0, idx].set_ylabel(metric_name)
        axes3[0, idx].grid(True, alpha=0.3)

        # Добавление значений
        for i, (bar, value) in enumerate(zip(bars, [control_val, test_val])):
            axes3[0, idx].text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.001,
                              f'{value:.4f}', ha='center', va='bottom', fontweight='bold')

    # Дополнительные метрики
    additional_metrics = [
        ('ARPU', control_data['ARPU'], test_data['ARPU']),
        ('ARPPU', control_data['ARPPU'] if not np.isnan(control_data['ARPPU']) else 0,
                test_data['ARPPU'] if not np.isnan(test_data['ARPPU']) else 0),
        ('Avg Order Value', control_data['avg_order_value'] if not np.isnan(control_data['avg_order_value']) else 0,
                          test_data['avg_order_value'] if not np.isnan(test_data['avg_order_value']) else 0)
    ]

    for idx, (metric_name, control_val, test_val) in enumerate(additional_metrics):
        bars = axes3[1, idx].bar(['Control', 'Test'], [control_val, test_val],
                               color=['#1f77b4', '#ff7f0e'], alpha=0.7, edgecolor='black')
        axes3[1, idx].set_title(metric_name, fontweight='bold')
        axes3[1, idx].set_ylabel('Рубли')
        axes3[1, idx].grid(True, alpha=0.3)

        for i, (bar, value) in enumerate(zip(bars, [control_val, test_val])):
            axes3[1, idx].text(bar.get_x() + bar.get_width()/2., bar.get_height() + max([control_val, test_val])*0.01,
                              f'{value:.2f} руб.', ha='center', va='bottom', fontweight='bold')

    plt.tight_layout()
    plt.savefig('ab_test_results.png', dpi=300, bbox_inches='tight')
    plt.close()

    report_content.append("![Результаты А/Б теста](ab_test_results.png)\n")

    # 2.4. Детальный анализ результатов
    report_content.append("### 2.2. Статистический анализ\n")

    report_content.append("**Результаты статистических тестов:**")
    report_content.append(f"- CTR: z-score = {z_ctr:.4f}, p-value = {p_ctr:.4f} {'✓' if p_ctr < 0.05 else '✗'}")
    report_content.append(f"- CR: z-score = {z_cr:.4f}, p-value = {p_cr:.4f} {'✓' if p_cr < 0.05 else '✗'}")

    report_content.append("\n**Изменение ключевых метрик:**")
    ctr_change = (ctr_test / ctr_control - 1) * 100
    cr_change = (test_data['CR'] / control_data['CR'] - 1) * 100 if control_data['CR'] > 0 else 0
    arpu_change = (test_data['ARPU'] / control_data['ARPU'] - 1) * 100 if control_data['ARPU'] > 0 else 0

    report_content.append(f"- CTR: {ctr_control:.4f} → {ctr_test:.4f} ({ctr_change:+.2f}%)")
    report_content.append(f"- CR: {control_data['CR']:.4f} → {test_data['CR']:.4f} ({cr_change:+.2f}%)")
    report_content.append(f"- ARPU: {control_data['ARPU']:.2f} руб. → {test_data['ARPU']:.2f} руб. ({arpu_change:+.2f}%)")

    # 2.5. Анализ воронки
    report_content.append("\n### 2.3. Анализ воронки конверсии\n")

    funnel_data = {
        'Этап': ['Просмотр главной', 'Клик по рекламе', 'Добавление в корзину', 'Заказ'],
        'Control': [
            control_data['client_id'],
            control_data['is_view_ads'],
            control_data['is_adds_ads'],
            control_data['is_orders_ads']
        ],
        'Test': [
            test_data['client_id'],
            test_data['is_view_ads'],
            test_data['is_adds_ads'],
            test_data['is_orders_ads']
        ]
    }

    funnel_df = pd.DataFrame(funnel_data)
    funnel_df['Конверсия Control'] = funnel_df['Control'] / funnel_df['Control'].iloc[0]
    funnel_df['Конверсия Test'] = funnel_df['Test'] / funnel_df['Test'].iloc[0]

    report_content.append(funnel_df.to_markdown(index=False))

    # Визуализация 4: Воронка конверсии
    fig4, (ax4_1, ax4_2) = plt.subplots(1, 2, figsize=(15, 6))

    # Воронка количеств
    stages = funnel_df['Этап']
    control_counts = funnel_df['Control']
    test_counts = funnel_df['Test']

    y_pos = np.arange(len(stages))

    ax4_1.barh(y_pos - 0.2, control_counts, 0.4, label='Control', color='#1f77b4', alpha=0.7)
    ax4_1.barh(y_pos + 0.2, test_counts, 0.4, label='Test', color='#ff7f0e', alpha=0.7)
    ax4_1.set_yticks(y_pos)
    ax4_1.set_yticklabels(stages)
    ax4_1.invert_yaxis()
    ax4_1.set_xlabel('Количество пользователей')
    ax4_1.set_title('Воронка конверсии по количеству', fontweight='bold')
    ax4_1.legend()
    ax4_1.grid(True, alpha=0.3)

    # Добавление значений на бары
    for i, (c_val, t_val) in enumerate(zip(control_counts, test_counts)):
        ax4_1.text(c_val + max(control_counts)/100, i - 0.2, f'{c_val:,}', va='center', fontweight='bold')
        ax4_1.text(t_val + max(test_counts)/100, i + 0.2, f'{t_val:,}', va='center', fontweight='bold')

    # Воронка конверсий
    control_conv = funnel_df['Конверсия Control']
    test_conv = funnel_df['Конверсия Test']

    ax4_2.barh(y_pos - 0.2, control_conv, 0.4, label='Control', color='#1f77b4', alpha=0.7)
    ax4_2.barh(y_pos + 0.2, test_conv, 0.4, label='Test', color='#ff7f0e', alpha=0.7)
    ax4_2.set_yticks(y_pos)
    ax4_2.set_yticklabels(stages)
    ax4_2.invert_yaxis()
    ax4_2.set_xlabel('Доля от начальной аудитории')
    ax4_2.set_title('Воронка конверсии (нормированная)', fontweight='bold')
    ax4_2.legend()
    ax4_2.grid(True, alpha=0.3)

    # Добавление значений на бары
    for i, (c_val, t_val) in enumerate(zip(control_conv, test_conv)):
        ax4_2.text(c_val + 0.01, i - 0.2, f'{c_val:.1%}', va='center', fontweight='bold')
        ax4_2.text(t_val + 0.01, i + 0.2, f'{t_val:.1%}', va='center', fontweight='bold')

    plt.tight_layout()
    plt.savefig('conversion_funnel.png', dpi=300, bbox_inches='tight')
    plt.close()

    report_content.append("\n![Воронка конверсии](conversion_funnel.png)\n")

    # Заключение и рекомендации
    report_content.append("## Заключение и рекомендации\n")

    # Определение результата
    is_significant = p_ctr < 0.05
    is_positive = ctr_change > 0

    if is_significant and is_positive:
        conclusion = "✅ **ЭКСПЕРИМЕНТ УСПЕШЕН**"
        recommendation = f"Изменение расположения рекламного блока показало статистически значимое улучшение CTR на {ctr_change:+.2f}%. Рекомендуется внедрить изменение для всех пользователей."
    elif is_significant and not is_positive:
        conclusion = "❌ **ЭКСПЕРИМЕНТ НЕУДАЧЕН**"
        recommendation = f"Изменение оказало негативное влияние на ключевую метрику (CTR уменьшился на {abs(ctr_change):.2f}%). Не рекомендуется внедрять данное изменение."
    else:
        conclusion = "⚠️ **РЕЗУЛЬТАТЫ НЕОПРЕДЕЛЕННЫ**"
        recommendation = "Эксперимент не показал статистически значимого влияния на ключевую метрику. Рекомендуется продлить тестирование или рассмотреть альтернативные гипотезы."

    report_content.append(f"### {conclusion}\n")
    report_content.append("### Рекомендация:\n")
    report_content.append(recommendation)

    report_content.append("\n### Дополнительные рекомендации:")
    report_content.append("1. Провести сегментный анализ для выявления групп пользователей, наиболее чувствительных к изменению")
    report_content.append("2. Оценить долгосрочное влияние на retention и LTV")
    report_content.append("3. Проанализировать возможное влияние на другие бизнес-метрики")
    report_content.append("4. Рассчитать ROI от внедрения изменения")

    # Сохранение отчета
    with open('finishdeepseek.md', 'w', encoding='utf-8') as f:
        f.write('\n'.join(report_content))

    print("✅ Отчет успешно создан: finishdeepseek.md")
    print("✅ Визуализации сохранены: historical_metrics.png, mde_analysis.png, ab_test_results.png, conversion_funnel.png")

    return report_content

# Запуск создания отчета
if __name__ == "__main__":
    report = create_final_report()
    print("\n" + "="*50)
    print("ФИНАЛЬНЫЙ ОТЧЕТ УСПЕШНО СОЗДАН!")
    print("="*50)

📁 Текущая рабочая директория: c:\ZC\finish\notebooks\DeepSeek
📁 Базовая директория проекта: c:\ZC\finish
📁 Путь к данным: c:\ZC\finish\Data\data_raw.csv
Столбцы в данных: ['Unnamed: 0', 'client_id', 'dt', 'event_type']
Типы событий: ['clicks' 'adds' 'views']
✅ Отчет успешно создан: finishdeepseek.md
✅ Визуализации сохранены: historical_metrics.png, mde_analysis.png, ab_test_results.png, conversion_funnel.png

ФИНАЛЬНЫЙ ОТЧЕТ УСПЕШНО СОЗДАН!
